In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('mydb.db')

In [3]:
connection

In [4]:
table_creation_query = """
CREATE TABLE IF NOT EXISTS employees(
    emp_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    hire_date TEXT NOT NULL,
    salary REAL NOT NULL
);
"""

In [5]:
table_creation_query2 = """
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone TEXT
);
"""

In [6]:
table_creation_query3 = """
CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    amount REAL NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
"""

In [7]:
cursor = connection.cursor()

In [8]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [9]:
insert_query_employees = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?)
"""

insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone)
VALUES (?, ?, ?, ?, ?)
"""


insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?)
"""

In [10]:
employee_data = [
    (1, "employee1", "last1", "employee1@gmail.com", "2021-01-01", 10000.00),
    (2, "employee2", "last2", "employee2@gmail.com", "2022-02-01", 20000.00),
    (3, "employee3", "last3", "employee3@gmail.com", "2023-03-01", 30000.00),
    (4, "employee4", "last4", "employee4@gmail.com", "2024-04-01", 40000.00),
]


customers_data = [
    (1, "king1", "last1", "king1@gmail.com", "112345678"),
    (2, "king2", "last2", "king2@gmail.com", "122345678"),
    (3, "king3", "last3", "king3@gmail.com", "123345678"),
    (4, "king4", "last4", "king4@gmail.com", "123445678"),
    
]


orders_data = [
    (1, 1, "2023-12-01", 250.00),
    (2, 2, "2023-23-01", 150.00),
    (3, 3, "2023-11-01", 300.00),
    (4, 4, "2023-04-03", 450.00),
]

In [11]:
cursor.executemany(insert_query_employees, employee_data)
cursor.executemany(insert_query_customers, customers_data)
cursor.executemany(insert_query_orders, orders_data)

IntegrityError: UNIQUE constraint failed: employees.emp_id

In [ ]:
connection.commit()

In [ ]:
cursor.execute("select * from orders")

In [ ]:
for row in cursor.fetchall():
    print(row)

(1, 1, '2023-12-01', 250.0)
(2, 2, '2023-23-01', 150.0)
(3, 3, '2023-11-01', 300.0)
(4, 4, '2023-04-03', 450.0)


In [13]:
from langchain_community.utilities import SQLDatabase

In [16]:
db = SQLDatabase.from_uri('sqlite:///mydb.db')

In [17]:
db

In [18]:
db.dialect

'sqlite'

In [19]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

In [22]:
llm.invoke("how are you?").content

"I'm just a language model, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?"

In [23]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [24]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [26]:
tools = toolkit.get_tools()

In [27]:
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [28]:
list_tables_tool = next((tool for tool in tools if tool.name=="sql_db_list_tables"), None)

In [29]:
list_tables_tool

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x72fe1e60b010>)

In [30]:
get_schema_tool = next((tool for tool in tools if tool.name=="sql_db_schema"), None)

In [31]:
get_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x72fe1e60b010>)

In [32]:
list_tables_tool.invoke("")

'customers, employees, orders'

In [35]:
print(get_schema_tool.invoke("customers"))


CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone
1	king1	last1	king1@gmail.com	112345678
2	king2	last2	king2@gmail.com	122345678
3	king3	last3	king3@gmail.com	123345678
*/
